# Can ESG generate Alpha

In [1]:
import lseg.data as ld
import pandas as pd
import plotly.offline as offline
from plotly import graph_objs as go
from IPython.display import display
# offline.offline.init_notebook_mode(connected=True)

In [2]:
ld.open_session()

<lseg.data.session.Definition object at 0x1f89e9da250 {name='workspace'}>

In [3]:
start_year = 2014
end_year = 2020
index = '.SPX'

### Get the performance data for the index benchmark

In [4]:
sp = ld.get_data(index, fields=['TR.CLOSEPRICE.date', 'TR.CLOSEPRICE'], parameters={'SDate': '{}-01-01'.format(start_year), 'EDate': '{}-01-01'.format(end_year), 'Frq':'CQ'})
sp['Rebased'] = sp['Close Price'] * 100 / sp['Close Price'][0]
display(sp.head())
display('Count: {}'.format(len(sp)))

,Instrument,Date,Close Price,Rebased
0,.SPX,2014-03-31,1872.34,100.0
1,.SPX,2014-06-30,1960.23,104.694126
2,.SPX,2014-09-30,1972.29,105.33824
3,.SPX,2014-12-31,2058.9,109.964002
4,.SPX,2015-03-31,2067.89,110.44415


'Count: 24'

In [5]:
def getSubset(dFrame):
    # filter out the companies with low ESG score at the begining year
    # meanScore = dFrame.mean()['ESG Score']
    dFrame = dFrame[dFrame['ESG Score'] > 70]
    return dFrame['Instrument'].tolist()

In [6]:
def getDataForYear(year):
    display('Getting data for: {}'.format(year))
    # get index constituents at the begining year
    df = ld.get_data('{}{}({}-01-01)'.format('0#', index, year), fields=['TR.TRESGScore'], parameters={'SDate': '{}-01-01'.format(year), 'Period': 'FY0'})
    # filter out the instruments based on ESG ratings
    subset = getSubset(df)
    # get the performance data for this subset
    df2 = ld.get_data(subset, fields=['TR.CLOSEPRICE.date', 'TR.CLOSEPRICE'], parameters={'SDate': '{}-01-01'.format(year), 'EDate': '{}-01-01'.format(year + 1), 'Frq':'CQ'})
    df2 = df2.dropna()
    # consolidate the price data for instruments
    alphaList = []
    ser = df2['Date'].value_counts()
    for x in ser[ser > 5].index:
        ser = df2.loc[df2['Date'] == x]['Close Price'].sum()
        alphaList.append({'Date': x, 'Close Price': ser})
    return alphaList


### Get the price performance of equal weighted, high ESG index constituents

In [7]:
dl3 = []
for x in range(start_year, end_year):
    ret = getDataForYear(x)
    dl3.extend(ret)

df3 = pd.DataFrame(dl3)    
df3 = df3.sort_values(by=['Date'])
df3 = df3.reset_index(drop=True)
df3['Rebased'] = df3['Close Price'] * 100 / df3['Close Price'][0]
display(df3.head())
display('Count: {}'.format(len(df3)))

'Getting data for: 2014'

c:\Users\gsbal\envs\rdlib\lib\site-packages\lseg\data\_tools\_dataframe.py:192: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



'Getting data for: 2015'

c:\Users\gsbal\envs\rdlib\lib\site-packages\lseg\data\_tools\_dataframe.py:192: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



'Getting data for: 2016'

...

c:\Users\gsbal\envs\rdlib\lib\site-packages\lseg\data\_tools\_dataframe.py:192: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



'Getting data for: 2019'

c:\Users\gsbal\envs\rdlib\lib\site-packages\lseg\data\_tools\_dataframe.py:192: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,Date,Close Price,Rebased
0,2014-03-31,4882.471746,100.000000
1,2014-06-30,5143.785154,105.352072
2,2014-09-30,5018.967420,102.795627
3,2014-12-31,5166.596010,105.819271
4,2015-03-31,5389.556948,110.385830


'Count: 24'

### Plot the comparison

In [8]:
# plot the graphs together
res = [go.Scatter(x=sp['Date'], y=sp['Rebased'], name='Index', mode='lines'),
       go.Scatter(x=df3['Date'], y=df3['Rebased'], name='ESG List', mode='lines')]
lay = {'shapes': [
        {'type': 'line','x0': '{}-01-01'.format(start_year),'y0': 100,'x1': '{}-01-01'.format(end_year),'y1': 100,'line': {'width': 1},}
    ],
    'showlegend': True
}
fig = go.Figure(data=res, layout=lay)
fig.update_layout(title='Comparison')
fig.show()